In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle
import requests
import os
import random
from fake_useragent import UserAgent
from itertools import cycle

In [ ]:
# Getting proxies
proxies = pd.read_csv('/Users/macbook/Downloads/Webshare 100 proxies.txt')
print('Number of proxies:', len(proxies))
proxies = proxies['Proxy'].values
proxy_pool = cycle(proxies)

### Supporting functions

In [ ]:
def start_browsing():
    
    ua = UserAgent()
    user_agent = ua.random
    proxy = next(proxy_pool)

    webdriver.DesiredCapabilities.CHROME['proxy'] = {
        "httpProxy": proxy,
        "ftpProxy": proxy,
        "sslProxy": proxy,
        "proxyType": "MANUAL"
    }
    chrome_options = webdriver.ChromeOptions()
    chrome_options.headless=False
    chrome_options.add_argument('--user-agent=%s' % user_agent)
    
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_experimental_option("prefs", {"profile.default_content_settings.cookies": 2})
    path = '/Users/macbook/Downloads/Chrome Driver'
    browser =  webdriver.Chrome(os.path.join(path, 'chromedriver'),chrome_options=chrome_options)
    browser.maximize_window()
    
    
    url = 'https://www.allianz.de/gesundheit/tierkrankenversicherung/rechner/#/'
    browser.get(url)

    # Cookies
    path = '/html/body/div[1]/div[2]/div[4]/div[2]/div/button'
    element = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, path)))
    element.click()

    # Input ZIP
    path = '//*[@formcontrolname="zip"]'
    element = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, path)))
    element.clear()
    #browser.find_element_by_xpath(path).clear()
    sleep(0.3)
    browser.find_element_by_xpath(path).send_keys('99998')
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)


    sleep(0.3)
    # Street
    path = '//*[@formcontrolname="street"]'
    browser.find_element_by_xpath(path).click()
    browser.find_element_by_xpath(path).send_keys(Keys.ARROW_DOWN)
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)


    sleep(0.3)
    # House number
    path = '//*[@formcontrolname="streetNumber"]'
    browser.find_element_by_xpath(path).send_keys('1')
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)

    sleep(0.7)  
    # Input birthdate
    path = '//*[@formcontrolname="birthdate"]'
    browser.find_element_by_xpath(path).send_keys('01.12.1990')
    sleep(0.7) 

    # Choose pet
    path = '//*[@formcontrolname="animalCategory"]'
    browser.find_element_by_xpath(path).click()
    sleep(0.4)
    browser.find_element_by_xpath(path).send_keys(Keys.ARROW_DOWN)
    sleep(0.2)
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)
    
    if browser.find_element_by_xpath(path).text == 'Meine Katze':
        pass
    else:
        print('Error when choosing an animal')
        browser.quit()

    sleep(0.7) 
    # Birthdate
    path = '//*[@formcontrolname="animalBirthDate"]'
    try:
        browser.find_element_by_xpath(path).send_keys('01.12.2020')
    except:
        pass
    sleep(0.2)
    browser.find_element_by_xpath(path).send_keys('01.12.2020')
    sleep(0.3)
    browser.find_element_by_xpath(path).send_keys(Keys.ESCAPE)
    sleep(0.3)
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)

    sleep(0.5)
    # Wurde Ihr Tier kastriert oder sterilisiert?
    # Choosing 'Ja'
    path = '//*[@formcontrolname="sterilized"]//nx-radio[1]'
    browser.find_element_by_xpath(path).click()

    sleep(0.5)
    # Wie wird Ihre Katze gehalten?
    # Choosing "ausschließlich in der Wohnung"
    path = '//*[@formcontrolname="catHousingType"]//nx-radio[1]'
    browser.find_element_by_xpath(path).click()

    sleep(0.5)
    # War oder ist Ihr Tier wegen Krankheiten, Fehlentwicklungen oder Unfällen beim Tierarzt in Behandlung?
    # Choosing "Nein"
    path = '//*[@formcontrolname="preExistingDiagnosis"]//nx-radio[2]'
    browser.find_element_by_xpath(path).click()
    
    return browser

In [ ]:
def input_zip_code(zip_code):
    # Input ZIP
    path = '//*[@formcontrolname="zip"]'
    browser.find_element_by_xpath(path).clear()
    sleep(0.3)
    browser.find_element_by_xpath(path).send_keys(zip_code)
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)


    sleep(0.3)
    # Street
    path = '//*[@formcontrolname="street"]'
    element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
    element.click()
    browser.find_element_by_xpath(path).send_keys(Keys.ARROW_DOWN)
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)


    sleep(0.2)
    # House number
    path = '//*[@formcontrolname="streetNumber"]'
    browser.find_element_by_xpath(path).send_keys('1')
    browser.find_element_by_xpath(path).send_keys(Keys.ENTER)

### Getting ZIP codes

In [ ]:
origin_df = pd.read_excel('/Users/macbook/Downloads/ZIP Code-Table 1.xlsx')
origin_df.columns = origin_df.iloc[1].values
origin_df = origin_df.drop([0,1]).reset_index(drop=True)
origin_df['ZIP CODE'] = origin_df['ZIP CODE'].apply(lambda x: '0' + str(x) if len(str(x))==4 else x)
origin_df['ZIP CODE'].nunique()

In [ ]:
zip_prices = pd.DataFrame(columns=['ZIP'])
missed_zips = [x for x in origin_df['ZIP CODE'].unique() if x not in zip_prices['ZIP'].unique()]
print(len(missed_zips))

### Looping through zip codes

In [ ]:
browser = start_browsing()
step = 0
batch = 8
waiting_time = 30

while len(missed_zips)>0:
    for i, zip_code in enumerate(missed_zips[1:]):
        
        print('\nStep', step%batch+1, '| ZIP:', zip_code, '| Browser Run:', step//batch+1)
        
        
        # Rotating the IP at every batch's request
        step+=1
        if step%batch == 0:
            browser.quit()
            
            try:
                browser = start_browsing()
                print('\nNew Browser Session')
            except:
                continue

        '''
        Inputting zip code data
        '''
        try:
            input_zip_code(zip_code)
        except:
            print('ZIP CODE ERROR')
            continue


        '''
        Getting prices
        '''
        try:
            
            # Waiting until the price is presented
            
            path = '//*[@id="product_TM001"]'
            element = WebDriverWait(browser, waiting_time).until(EC.visibility_of_element_located((By.XPATH, path)))
            
            path = '/html/body/div[3]/div/div[4]/div/div/div/div/div/app-root/div/page-offer/div/div[2]/div/module-table/div/nx-comparison-table/div/nx-comparison-table-flex-row[1]/div[2]'
            header_price = browser.find_element_by_xpath(path).text
            
            basic, smart, komfort, premium = [header_price.split('\n')[i] for i in [1,5,9,13]]


            page_df = pd.DataFrame({'Basic':basic, 
                                    'Smart':smart,
                                    'Komfort':komfort,
                                    'Premium':premium}, index=[0])

            page_df['ZIP'] = zip_code
            zip_prices = pd.concat([zip_prices, page_df])
            display(page_df) # Delete this if you don't want to display table for every zip code
        
        except:
            print('PAGE ERROR')
            browser.back()
            continue


        # Back to input data
        browser.back()

    
    missed_zips = [x for x in origin_df['ZIP CODE'].unique() if x not in zip_prices['ZIP'].unique()] 
    random.shuffle(missed_zips)
    
    print('\nNumber of missed ZIPs',len(missed_zips), '| ZIP processed:', zip_prices['ZIP'].nunique())
    
    # Save the dataset
    #zip_prices.to_csv('ALIANZ_ZIP_DATA.csv')